In [1]:
import torch

In [3]:
torch.arange(2)

tensor([0, 1])

In [5]:
x = torch.rand((2,5))
x

tensor([[0.2450, 0.4297, 0.8310, 0.1996, 0.6158],
        [0.8604, 0.2999, 0.8781, 0.1361, 0.3998]])

In [ ]:
y = torch 